In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [18]:
import argparse
import glob
import json
import os
from pathlib import Path

import numpy as np
import torch
import yaml
from tqdm import tqdm

from utils.google_utils import attempt_load
from utils.datasets import create_dataloader
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, box_iou, \
    non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, clip_coords, set_logging, increment_path
from utils.loss import compute_loss
from utils.metrics import ap_per_class
from utils.plots import plot_images, output_to_target
from utils.torch_utils import select_device, time_synchronized

from models.models import *

In [19]:
def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

In [20]:
batch_size=16,
imgsz=640,
conf_thres=0.5,
iou_thres=0.6,  # for NMS
save_json=True,
single_cls=False,
augment=False,
verbose=True,
model=None,
dataloader=None,
save_dir=Path(''),  # for saving images
save_txt=False,  # for auto-labelling
save_conf=False,
plots=True,
log_imgs=0  # number of logged images
task = "test"

In [21]:
# You must run this one, the follwoing code relys on this opt object
from argparse import Namespace
opt = Namespace(augment=False, batch_size=32, cfg='./cfg/yolor_p6.cfg', conf_thres=0.5, data='../zero-waste-1/data.yaml', device='0', exist_ok=False, img_size=416, iou_thres=0.65, name='exp', names='./data/zerowaste.names', project='../runs/test', save_conf=True, save_json=True, save_txt=False, single_cls=False, task='test', verbose=True, weights=['../runs/train/yolor_p6_2022_03_26-10_44_07/weights/best_overall.pt'])

# ===========================> The following are some params will be passed to test() funciton
weights=["../runs/train/yolor_p6_2022_03_26-10_44_07/weights/best_overall.pt"]
data = "../zero-waste-1/data.yaml"	# Because this file will load some utils module, so this file must be put under yolor folder
batch_size=32

imgsz=640
# conf_thres=0.5	# object confidence threshold
# iou_thres=0.6  # IOU threshold for NMS, or .65
task='test'
device='0'	# help='cuda device, i.e. 0 or 0,1,2,3 or cpu'
single_cls=False
augment=False
verbose=True
# save_txt=False	# for auto-labelling
# save_conf=True
save_json=True

project='../runs/test'
name='exp'
exist_ok=False
cfg='./cfg/yolor_p6.cfg'
names='./data/zerowaste.names'

# model=None
log_imgs=0  	# number of logged imag
save_dir=Path('./')  # for saving images
plots=True
dataloader=None

In [22]:
save_dir

WindowsPath('.')

In [23]:
set_logging()
device = select_device(opt.device, batch_size=opt.batch_size)
save_txt = opt.save_txt  # save *.txt labels

# Directories: where all the logging file will be saved for this experiment
save_dir = Path(increment_path(Path(opt.project) / opt.name, exist_ok=opt.exist_ok))  # increment run
(save_dir / 'labels' if save_txt else save_dir).mkdir(parents=True, exist_ok=True)  # make dir

# Load model
model = Darknet(opt.cfg).to(device)

Using torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 8191MB)

Model Summary: 665 layers, 36854616 parameters, 36854616 gradients


In [24]:
try:
    ckpt = torch.load(opt.weights[0], map_location=device)  # load checkpoint
    ckpt['model'] = {k: v for k, v in ckpt['model'].items() if model.state_dict()[k].numel() == v.numel()}
    model.load_state_dict(ckpt['model'], strict=False)
except:
    load_darknet_weights(model, opt.weights[0])
imgsz = check_img_size(imgsz, s=64)  # check img_size

# Half: half precision if we are running on GPU
half = device.type != 'cpu'
if half:
    model.half()



In [25]:
# Configure
yaml_dir='../zero-waste-1/data.yaml'
model.eval()
is_coco = yaml_dir.endswith('coco.yaml')  # is COCO dataset
with open(yaml_dir) as f:
    data = yaml.load(f, Loader=yaml.FullLoader)  # model dict


In [26]:
print(data)	# Making sure you are running this file under yolor folder

{'names': ['cardboard', 'metal', 'rigid_plastic', 'soft_plastic'], 'nc': 4, 'train': '../zero-waste-1/train/images', 'val': '../zero-waste-1/valid/images', 'test': '../zero-waste-1/test/images'}


In [27]:
check_dataset(data)  # check
nc = 1 if single_cls else int(data['nc'])  # number of classes
iouv = torch.linspace(0.5, 0.95, 10).to(device)  # iou vector for mAP@0.5:0.95
niou = iouv.numel()

In [28]:
# Logging
log_imgs, wandb = min(log_imgs, 100), None  # ceil
try:
    import wandb  # Weights & Biases
except ImportError:
    log_imgs = 0

# Dataloader
img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
_ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
path = data['test'] if opt.task == 'test' else data['val']  # path to val/test images
dataloader = create_dataloader(path, imgsz, batch_size, 64, opt, pad=0.5, rect=True)[0]

C:\Users\59384\anaconda3\envs\latest_tf\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Scanning labels ..\zero-waste-1\test\labels.cache3 (8 found, 0 missing, 0 empty, 0 duplicate, for 8 images): 8it [00:00, 1599.81it/s]


In [29]:
path

'../zero-waste-1/test/images'

In [30]:
seen = 0
try:
    names = model.names if hasattr(model, 'names') else model.module.names
except:
    names = load_classes(opt.names)

In [31]:
names

['cardboard', 'metal', 'rigid_plastic', 'soft_plastic']

In [32]:
coco91class = coco80_to_coco91_class()  # Return a int list, has value from 1 to 90
s = ('%20s' + '%12s' * 6) % ('Class', 'Images', 'Targets', 'P', 'R', 'mAP@.5', 'mAP@.5:.95')    # ==> '               Class      Images     Targets           P           R      mAP@.5  mAP@.5:.95'
p, r, f1, mp, mr, map50, map, t0, t1 = 0., 0., 0., 0., 0., 0., 0., 0., 0.
loss = torch.zeros(3, device=device)
jdict, stats, ap, ap_class, wandb_images = [], [], [], [], []
"""
    jdict: json dictionary, what we use to save as json file, for futher evaluation
"""

'\n    jdict: json dictionary, what we use to save as json file, for futher evaluation\n'

In [33]:
dir(dataloader)[-20:]

['_get_iterator',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'iterator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [35]:
training = False
for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader, desc=s)):
    img = img.to(device, non_blocking=True)
    img = img.half() if half else img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    targets = targets.to(device)
    nb, _, height, width = img.shape  # batch size, channels, height, width
    whwh = torch.Tensor([width, height, width, height]).to(device)
    # Disable gradients
    with torch.no_grad():
        # Run model
        t = time_synchronized()
        inf_out, train_out = model(img, augment=augment)  # inference and training outputs
        t0 += time_synchronized() - t
        # Compute loss
        if training:  # if model has loss hyperparameters
            loss += compute_loss([x.float() for x in train_out], targets, model)[1][:3]  # box, obj, cls
        # Run NMS
        t = time_synchronized()
        output = non_max_suppression(inf_out, conf_thres=conf_thres, iou_thres=iou_thres)
        t1 += time_synchronized() - t
    # Statistics per image
    for si, pred in enumerate(output):  # si is index, pred is the predicted bbox, e.g., [[269.00000, 146.25000, 328.50000, 285.00000,   0.68604,   0.00000], so [x, y, w, h, score, category]
        labels = targets[targets[:, 0] == si, 1:]
        nl = len(labels)
        tcls = labels[:, 0].tolist() if nl else []  # target class
        seen += 1
        if len(pred) == 0:
            if nl:
                stats.append((torch.zeros(0, niou, dtype=torch.bool), torch.Tensor(), torch.Tensor(), tcls))
            continue
        # Append to text file
        path = Path(paths[si])
        if save_txt:
            gn = torch.tensor(shapes[si][0])[[1, 0, 1, 0]]  # normalization gain whwh
            x = pred.clone()
            x[:, :4] = scale_coords(img[si].shape[1:], x[:, :4], shapes[si][0], shapes[si][1])  # to original
            for *xyxy, conf, cls in x:
                xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
                with open(save_dir / 'labels' / (path.stem + '.txt'), 'a') as f:
                    f.write(('%g ' * len(line)).rstrip() % line + '\n')
        # W&B logging
        if plots and len(wandb_images) < log_imgs:
            box_data = [{"position": {"minX": xyxy[0], "minY": xyxy[1], "maxX": xyxy[2], "maxY": xyxy[3]},
                         "class_id": int(cls),
                         "box_caption": "%s %.3f" % (names[int(cls)], conf),
                         "scores": {"class_score": conf},
                         "domain": "pixel"} for *xyxy, conf, cls in pred.tolist()]
            boxes = {"predictions": {"box_data": box_data, "class_labels": names}}
            # print(f"img[si]: {img[si]}, boxes: {boxes}, path.name: {path.name}")
            # print(f"wandb.Image(img[si], boxes=boxes, caption=path.name): {wandb.Image(img[si], boxes=boxes, caption=path.name)}")
            wandb_images.append(wandb.Image(img[si], boxes=boxes, caption=path.name))
        # Clip boxes to image bounds
        clip_coords(pred, (height, width))
        # Append to pycocotools JSON dictionary
        if save_json:
            # [{"image_id": 42, "category_id": 18, "bbox": [258.15, 41.29, 348.26, 243.78], "score": 0.236}, ...
            image_id = int(path.stem) if path.stem.isnumeric() else path.stem
            # Note, if we don't get numeric() value, that it's taking filename, which will give us something like, "image_id": "12_frame_024600_PNG.rf.7f0302039ef984cd4bcdc54cacdc48fd" --> This is bad, and it can't be evaluate with cocoAPI  ==> Results do not correspond to current coco set
            box = pred[:, :4].clone()  # xyxy
            scale_coords(img[si].shape[1:], box, shapes[si][0], shapes[si][1])  # to original shape
            box = xyxy2xywh(box)  # xywh
            box[:, :2] -= box[:, 2:] / 2  # xy center to top-left corner
            for p, b in zip(pred.tolist(), box.tolist()):
                jdict.append({'image_id': image_id,
                              'category_id': coco91class[int(p[5])] if is_coco else int(p[5]),
                              'bbox': [round(x, 3) for x in b],
                              'score': round(p[4], 5)})
        # Assign all predictions as incorrect
        correct = torch.zeros(pred.shape[0], niou, dtype=torch.bool, device=device)
        if nl:
            detected = []  # target indices
            tcls_tensor = labels[:, 0]
            # target boxes
            tbox = xywh2xyxy(labels[:, 1:5]) * whwh
            # Per target class
            for cls in torch.unique(tcls_tensor):
                ti = (cls == tcls_tensor).nonzero(as_tuple=False).view(-1)  # prediction indices
                pi = (cls == pred[:, 5]).nonzero(as_tuple=False).view(-1)  # target indices
                # Search for detections
                if pi.shape[0]:
                    # Prediction to target ious
                    ious, i = box_iou(pred[pi, :4], tbox[ti]).max(1)  # best ious, indices
                    # Append detections
                    detected_set = set()
                    for j in (ious > iouv[0]).nonzero(as_tuple=False):
                        d = ti[i[j]]  # detected target
                        if d.item() not in detected_set:
                            detected_set.add(d.item())
                            detected.append(d)
                            correct[pi[j]] = ious[j] > iouv  # iou_thres is 1xn
                            if len(detected) == nl:  # all targets already located in image
                                break
        # Append statistics (correct, conf, pcls, tcls)
        stats.append((correct.cpu(), pred[:, 4].cpu(), pred[:, 5].cpu(), tcls))
    # Plot images
    if plots and batch_i < 3:
        print(f"=========> type(output): {type(output)}")
        output = [i.cpu().detach().numpy()  for i in output]
        f = save_dir / f'test_batch{batch_i}_labels.jpg'  # filename
        plot_images(img, targets, paths, f, names)  # labels
        f = save_dir / f'test_batch{batch_i}_pred.jpg'
        plot_images(img, output_to_target(output , width, height), paths, f, names)  # predictions
        # f = save_dir
        # plot_results(save_dir=save_dir)  # predictions

               Class      Images     Targets           P           R      mAP@.5  mAP@.5:.95:   0%|          | 0/1 [00:00<?, ?it/s]


TypeError: '>' not supported between instances of 'Tensor' and 'tuple'

In [9]:
anno_json = glob.glob("/projectnb/dl523/students/dong760/roboflow-ai/test/_annotations.coco.json")[0]
# importing the module
pred_json = "../runs/test/exp43/best_overall_predictions.json"
import json
 
# Opening JSON file
f1 = open(anno_json)
f2 = open(pred_json)
# returns JSON object as
# a dictionary
gT_dict = json.load(f1)
jdict = json.load(f2)




In [ ]:
converer_dict = gT_dict["images"]
suffix = converer_dict[0]["file_name"].split('.')[-1]
for i in range(len(jdict)):
    image_id = jdict[i]['image_id'] + "." + suffix
    if not image_id.isnumeric():
        flag = True # Mark as incorrect image_id
    for item in converer_dict:
        # If we found a match, and the current image_id is marked as incorrect format ==> We will correct it
        if flag and image_id == item["file_name"]:
            print(f"jdict[i]['image_id']: {jdict[i]['image_id']}, item['id']: {item['id']}")
            jdict[i]['image_id'] = item['id']

In [24]:
jdict[1]["image_id"].isnumeric()

'08_frame_011500_PNG.rf.fa2aa81dc2839b709ca7ce5e9feeed9b'

In [26]:

converer_dict[428]["file_name"].split('.')[-1]
# for item in converer_dict:
#     # If we found a match, and the current image_id is marked as incorrect format ==> We will correct it
#     if flag and image_id == item["file_name"]:
#         jdict[i]["image_id"] = item["id"]

'jpg'

In [31]:
pred_json = "../runs/test/exp51/best_overall_predictions.json"
# anno_json = glob.glob("coco/annotations/instances_val*.json")[0]  
anno_json = glob.glob("/projectnb2/dl523/students/dong760/zerowaste_dataset/zerowaste-f-final/test/labels.json")[0]  # finding the annotations json 
# anno_json = glob.glob("/projectnb/dl523/students/dong760/roboflow-ai/test/_annotations.coco.json")[0]  # finding the annotations json 
is_coco = False
try:  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPIpycocoEvalDemo.ipynb
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval
    anno = COCO(anno_json)  # init annotations api
    pred = anno.loadRes(pred_json)  # init predictions api
    # imgIds=sorted(anno.getImgIds())
    # imgIds=imgIds[0:100]
    # imgId = imgIds[np.random.randint(100)]
    eval = COCOeval(anno, pred, 'bbox')
    if is_coco:
        eval.params.imgIds = [int(Path(x).stem) for x in dataloader.dataset.img_files]  # image IDs to evaluate
    eval.evaluate()
    eval.accumulate()
    eval.summarize()
    map, map50 = eval.stats[:2]  # update results (mAP@0.5:0.95, mAP@0.5)
    print("================> Start Printing evaluated result:")
    print(eval.stats)
    print("================> End of Printing!")
except Exception as e:
    print('ERROR: pycocotools unable to run: %s' % e)

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.45s).
Accumulating evaluation results...
DONE (t=0.11s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets

In [79]:
anno.getCatIds()

[1, 2, 3, 4]

In [3]:
#initialize COCO ground truth api
dataDir='../'
dataType='val2014'
annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=8.01s)
creating index...
index created!


In [4]:
#initialize COCO detections api
resFile='%s/results/%s_%s_fake%s100_results.json'
resFile = resFile%(dataDir, prefix, dataType, annType)
cocoDt=cocoGt.loadRes(resFile)

Loading and preparing results...     
DONE (t=0.05s)
creating index...
index created!


In [5]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [6]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...      
DONE (t=0.46s).
Accumulating evaluation results...   
DONE (t=0.38s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.697
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.586
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.387
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.566
 Average Rec